In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../src/data/interactions.csv')
df_users = pd.read_csv('../src/data/users.csv')
df_items = pd.read_csv('../src/data/items.csv')

In [118]:
df.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [139]:
df_users[df_users["user_id"] == 2].values.squeeze()

array([2, '18_24', 0.0], dtype=object)

In [126]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142888 entries, 0 to 142887
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  142888 non-null  int64  
 1   age      142742 non-null  object 
 2   sex      136626 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.3+ MB


In [127]:
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [6]:
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


# Cold Start

## Возьмем самое популярное

In [7]:
items_name = df_items[['id', 'title']].set_index('id').sort_index()

In [8]:
items_count = pd.DataFrame(df[['item_id']].value_counts(), columns=['count']).sort_index()

In [9]:
count_titles = items_name.merge(items_count, left_index=True, right_on='item_id')

In [10]:
count_titles.sort_values(by='count', ascending=False).head(10)

,title,count
item_id,,
99357,"Сила подсознания, или Как изменить жизнь за 4 ...",6576
316995,Самая таинственная тайна и другие сюжеты,2706
109201,Яблоки из сада Шлицбутера,2585
241026,Зелёная миля,2300
28889,Сказки для идиотов (сборник),2287
121687,Снеговик,2245
243711,Противостояние,2143
288531,Инферно,2138
90225,Зулейха открывает глаза,2098


## Теперь попробуем почистить датасет

In [11]:
items_count = pd.DataFrame(df[df['progress'] > 40][['item_id']].value_counts(), columns=['count']).sort_index() 

In [12]:
count_titles = items_name.merge(items_count, left_index=True, right_on='item_id')

In [13]:
count_titles.sort_values(by='count', ascending=False).head(10)

,title,count
item_id,,
90225,Зулейха открывает глаза,1071
281005,"Богатый папа, бедный папа. Роберт Кийосаки (об...",994
109201,Яблоки из сада Шлицбутера,946
99357,"Сила подсознания, или Как изменить жизнь за 4 ...",796
319390,Черновик,720
121687,Снеговик,699
264997,После,696
89152,50 правил умной дуры,595
159580,Пятьдесят оттенков серого,542


## Посмотрим какие жанры популярны

In [98]:
items_count = pd.DataFrame(df[df['progress'] > 40][['item_id']].value_counts(), columns=['count']).sort_index() 

In [99]:
items_name = df_items[['id', 'title', 'genres']].set_index('id').sort_index()

In [117]:
    items_count = pd.DataFrame(
            df[df["progress"] > 40][["item_id"]].value_counts(), columns=["count"]
    ).sort_index()
    items_name = df_items[['id', 'title', 'genres']].set_index('id').sort_index()
    items_name.genres = items_name.genres.fillna('Другие жанры')
    items_name.genres = items_name.genres.apply(lambda x: x.split(','))
    items_name_exlode = items_name.explode(column='genres')
    items_name_exlode = items_name_exlode[items_name_exlode['genres'] == 'Зарубежные приключения'][['title']]
    count_titles = items_name_exlode.merge(items_count, left_index=True, right_on='item_id')

count_titles.sort_values(by='count', ascending=False)['title'].values[:n]

array(['Шантарам', 'Тень горы', 'Таинственный остров', 'Повелитель мух',
       'Путешественник', 'Двадцать тысяч лье под водой',
       'Два года каникул', 'Песчаный дьявол', 'Женская война',
       'Граф Монте-Кристо', 'Плавающий город', 'Амазония',
       'Костяной лабиринт', 'Всадник без головы', 'Шестое вымирание',
       'Дети капитана Гранта', 'Седьмая казнь', 'Ледяные боги',
       'Крепкие мужчины', 'Пещера', 'Пять недель на воздушном шаре',
       'Мотылек', 'Айвенго', 'Приключения Тома Сойера', 'Сёгун',
       'Две Дианы', 'Белый клык', 'Сердца трех', 'Дети капитана Гранта',
       'Малыш', 'Тонкая работа', 'Первая ночь', 'Три мушкетера',
       'Дальнейшие приключения Робинзона Крузо', 'Черная Индия',
       'Королева Марго', 'Моби Дик, или Белый Кит', 'Песнь бича',
       'Графиня де Монсоро', 'Ким', 'Юные путешественники',
       'Королева Марго',
       'Ужас на поле для гольфа. Приключения Жюля де Грандена (сборник)',
       'Тайпан', 'Порочный круг', 'Айвенго', 'Влюбле

In [84]:
count_titles.genres = count_titles.genres.fillna('NaN')

In [85]:
count_titles.genres = count_titles.genres.apply(lambda x: x.split(','))

In [86]:
count_titles_exlode = count_titles.explode(column='genres') #.pivot_table(index='title', columns='genres')

In [92]:
count_titles_exlode[count_titles_exlode['genres'] == 'Зарубежные приключения'][['title', 'count']].sort_values(by='count', ascending=False)

,title,count
item_id,,
26963,Шантарам,462
306416,Тень горы,136
292066,Таинственный остров,112
198593,Повелитель мух,86
108078,Путешественник,59
...,...,...
284119,The Black Arrow,2
22209,Приключения бригадира Жерара,2
170237,В сердце моря. Трагедия китобойного судна «Эсс...,2


In [93]:
items_name

,title,genres
id,,
1,Множественные источники дохода,"Эффективность бизнеса,Финансовый менеджмент,Пр..."
2,"Она не объясняет, он не догадывается. Японское...","Семейная психология,Зарубежная психология,Соци..."
13,Падение,"Зарубежные любовные романы,Остросюжетные любов..."
21,Надо жить умеючи. Архетипы нового века,"Прикладная литература,Общая психология,Публици..."
26,Сетевой маркетинг: методы продаж,"Рукоделие и ремесла,Просто о бизнесе,Другие сп..."
...,...,...
321732,Молитвы на разные случаи,"Православные молитвы,Христианство"
321733,Доктор Кто. Герои и монстры (сборник),"Боевая фантастика,Зарубежная фантастика,Космич..."
321739,Человек в футляре,Современная русская литература
